## 25 ml - Create no context training data from context

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from joblib import dump
import pandas as pd
import numpy as np
import json

In [2]:
# depth of this filr in the project
file_depth = '../..'

with open(file_depth + '/config/data_25m_config.json') as config_file:
    config = json.load(config_file)

new_context_data = config['new_context_data']
context_data = pd.read_csv(file_depth + new_context_data)
context_data

,userId,movieId,day,isWeekday,season,partOfDay,holiday,movieYear,titleType,isAdult,...,genreMystery,genreNews,genreReality-tv,genreRomance,genreSci-fi,genreShort,genreSport,genreThriller,genreWar,genreWestern
0,1,296,3,1,1,3,no_holiday,1994,movie,0,...,0,0,0,0,0,0,0,1,0,0
1,1,306,3,1,1,2,no_holiday,1994,movie,0,...,0,0,0,0,0,0,0,0,0,0
2,1,307,3,1,1,2,no_holiday,1993,movie,0,...,0,0,0,0,0,0,0,0,0,0
3,1,665,3,1,1,3,no_holiday,1995,movie,0,...,0,0,0,0,0,0,0,0,1,0
4,1,899,3,1,1,2,no_holiday,1952,movie,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24983464,162541,50872,2,1,1,4,no_holiday,2007,movie,0,...,0,0,0,0,0,0,0,0,0,0
24983465,162541,55768,2,1,1,4,no_holiday,2007,movie,0,...,0,0,0,0,0,0,0,0,0,0
24983466,162541,56176,2,1,1,4,no_holiday,2007,movie,0,...,0,0,0,0,0,0,0,0,0,0
24983467,162541,58559,2,1,1,4,no_holiday,2008,movie,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
no_context_data = context_data[['userId', 'movieId']]
del context_data
no_context_data.reset_index(drop=True)
no_context_data

,userId,movieId
0,1,296
1,1,306
2,1,307
3,1,665
4,1,899
...,...,...
24983464,162541,50872
24983465,162541,55768
24983466,162541,56176
24983467,162541,58559


In [14]:
target_data = config['target_data']
target_ratings = pd.read_csv(file_depth + target_data)
target_ratings

,ratingId,rating
0,0,5.0
1,1,3.5
2,2,5.0
3,3,5.0
4,4,3.5
...,...,...
24983464,24983464,4.5
24983465,24983465,2.5
24983466,24983466,2.0
24983467,24983467,4.0


In [5]:
no_context_data['ratingId'] = context_data.index
del context_data
no_context_data = no_context_data.merge(target_ratings, on='ratingId', how='left')
del target_data
no_context_data = no_context_data.drop(['ratingId'], axis=1)
no_context_data

C:\Users\adamb\AppData\Local\Temp\ipykernel_452\1015003645.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_context_data['ratingId'] = context_data.index


,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5
...,...,...,...
24983464,162541,50872,4.0
24983465,162541,55768,4.0
24983466,162541,56176,4.0
24983467,162541,58559,4.0


In [8]:
print('Are three any NaN values? ', no_context_data.isna().any().any())

Are three any NaN values?  False


In [9]:
train, temp = train_test_split(no_context_data, test_size=0.2, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)

In [11]:
train_target_ratings = train['rating'].values
val_target_ratings = val['rating'].values
test_target_ratings = test['rating'].values

train_data = train.drop(['rating'], axis=1)
val_data = val.drop(['rating'], axis=1)
test_data = test.drop(['rating'], axis=1)

In [12]:
# Standardize columns
scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
val_data = scaler.fit_transform(val_data)
test_data = scaler.fit_transform(test_data)

In [13]:
# Save the data to disk

training_data_path = config['training_data']
data_group = 'data_25m/'
folder = 'no_context/'

dump(scaler, file_depth + training_data_path + f"{data_group}{folder}25m_no_context_scaler.pkl")
del scaler

np.save(file_depth + training_data_path + f"{data_group}{folder}train_data.npy", train_data)
del train_data
np.save(file_depth + training_data_path + f"{data_group}{folder}val_data.npy", val_data)
del val_data
np.save(file_depth + training_data_path + f"{data_group}{folder}test_data.npy", test_data)
del test_data
np.save(file_depth + training_data_path + f"{data_group}{folder}train_target_ratings.npy", train_target_ratings)
del train_target_ratings
np.save(file_depth + training_data_path + f"{data_group}{folder}val_target_ratings.npy", val_target_ratings)
del val_target_ratings
np.save(file_depth + training_data_path + f"{data_group}{folder}test_target_ratings.npy", test_target_ratings)